In [1]:
from math import isclose
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("D:/GitHub/apode/")  # ver
from apode import ApodeData

In [ ]:
# Notas
# ver comparacion relativa (cuando es cercano a cero usar absoluta)
# ver argumentos enteros, requiere reales para no dar error
# surgen infinitos (/0) en algunos casos extremos, ver tratamiento de RuntimeWarning
# gini tenia mal un indice
# wolfson: se cambio el algoritmo tal cual sale en ineq R (ahora da igual)

In [3]:
# n par
def make_data(n,tipo):
    if tipo==1:
        return np.ones(n)*10
    elif tipo==2:
        return list(range(1,n+1))
    elif tipo==3:
        return np.power(range(1,n+1),2)
    elif tipo==4:
        return np.concatenate((np.zeros(n-1),[10]))
    elif tipo==5:  
        n1 = int(n/2)  
        a = np.power(range(1,n1+1),1.2)
        b = n1^2
        y = np.concatenate((b-a, b+a))
        return  np.sort(y-min(y)+1)        
    elif tipo==6:  
        n1 = int(n/2)  
        a = np.power(range(1,n1+1),0.5)
        b = n1^2
        y = np.concatenate((b-a, b+a))
        return np.sort(y-min(y)+1)          

### Test en R

In [4]:
# listado de lineas maximas
fname = "test_ineq.csv" 
path = 'D:\\GitHub\\apode\\Drafts\\test_R\\'
# D:\GitHub\apode\apode\Drafts\test_R
#D:\GitHub\apode
dfR = pd.read_csv(fname) 
nf = dfR.shape[0]
n = 100 # ver en R
dfR

,gini,atkinson05,atkinson1,atkinson2,entropy0,entropy1,entropy2,kolm05,varcoeff,rosenbluth,wolfson
0,0.000000,0.000000,-2.220446e-16,0.000000,-2.220446e-16,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000
1,0.330000,0.107203,2.476695e-01,0.618266,2.845796e-01,0.188273,0.163366,42.155164,0.571605,0.014925,0.330099
2,0.497463,0.246269,5.733872e-01,0.981923,8.518784e-01,0.426942,0.395493,inf,0.889374,0.019899,0.660133
3,0.990000,0.990000,1.000000e+00,1.000000,inf,0.000000,49.500000,0.080035,9.949874,1.000000,inf
4,0.315046,0.099368,2.376409e-01,0.686454,2.713375e-01,0.172642,0.149354,100.756726,0.546542,0.014600,0.290637
5,0.353321,0.118546,2.490380e-01,0.483053,2.864003e-01,0.218827,0.195726,3.968104,0.625661,0.015464,0.478014


In [6]:
# ineq
for j in range(1,10):
    alpha = None
    if j==1:
        method = 'gini'
        valR = dfR.gini
    elif j==2:
        method = 'atkinson'
        valR = dfR.atkinson05
        alpha = 0.5
    elif j==3:
        method = 'atkinson'
        valR = dfR.atkinson1
        alpha = 1
    elif j==4:
        method = 'atkinson'
        valR = dfR.atkinson2
        alpha = 2
    elif j==5:
        method = 'entropy'
        valR = dfR.entropy0
        alpha = 0    
    elif j==6:
        method = 'entropy'
        valR = dfR.entropy1
        alpha = 1    
    elif j==7:
        method = 'entropy'
        valR = dfR.entropy2
        alpha = 2    
    elif j==8:
        method = 'kolm'
        valR = dfR.kolm05
        alpha = 0.5   
    elif j==9:
        method = 'cv'
        valR = dfR.varcoeff
        alpha = None           
        
    for i in range(1,nf+1):
        x = make_data(n,i) 
        df = pd.DataFrame({'x':x})+0.0  # si es entero tira error
        dr = ApodeData(df, varx="x")
        if alpha==None:
            g = dr.inequality(method)
        else:
            g = dr.inequality(method,alpha=alpha)
        print([method,alpha,valR[i-1], g, isclose(valR[i-1],g)])

['gini', None, 0.0, -2.19824158875781e-16, False]
['gini', None, 0.33, 0.3299999999999999, True]
['gini', None, 0.49746268656716397, 0.4974626865671641, True]
['gini', None, 0.99, 0.9899999999999999, True]
['gini', None, 0.315046152795281, 0.315046152795281, True]
['gini', None, 0.35332053566084004, 0.35332053566083266, True]
['atkinson', 0.5, 0.0, -2.220446049250313e-16, False]
['atkinson', 0.5, 0.10720299142090199, 0.10720299142090162, True]
['atkinson', 0.5, 0.246268656716418, 0.24626865671641796, True]
['atkinson', 0.5, 0.99, 0.99, True]
['atkinson', 0.5, 0.0993684870483649, 0.09936848704836476, True]
['atkinson', 0.5, 0.11854581360391801, 0.11854581360391192, True]
['atkinson', 1, -2.2204460492503099e-16, -2.220446049250313e-16, True]
['atkinson', 1, 0.247669517924073, 0.2476695179240732, True]
['atkinson', 1, 0.573387189699103, 0.5733871896991026, True]
['atkinson', 1, 1.0, -2.220446049250313e-16, False]
['atkinson', 1, 0.237640879091862, 0.2376408790918606, True]
['atkinson', 1,

D:/GitHub/apode\apode\inequality.py:74: RuntimeWarning: divide by zero encountered in power
  a1 = np.sum(np.power(y, 1 - alpha)) / n
D:/GitHub/apode\apode\inequality.py:52: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(np.log(u / y)) / n
D:/GitHub/apode\apode\inequality.py:54: RuntimeWarning: divide by zero encountered in log
  return np.sum((y / u) * np.log(y / u)) / n
D:/GitHub/apode\apode\inequality.py:54: RuntimeWarning: invalid value encountered in multiply
  return np.sum((y / u) * np.log(y / u)) / n
D:/GitHub/apode\apode\inequality.py:215: RuntimeWarning: overflow encountered in exp
  np.sum(np.exp(alpha * (u - y)))))


In [7]:
# concentration
for j in range(1,2):
    alpha = None
    if j==1:
        method = 'rosenbluth'
        valR = dfR.rosenbluth
            
    for i in range(1,nf+1):
        x = make_data(n,i) 
        df = pd.DataFrame({'x':x})+0.0  # si es entero tira error
        dr = ApodeData(df, varx="x")
        if alpha==None:
            g = dr.concentration(method)
        else:
            g = dr.concentration(method,alpha=alpha)
        print([method,alpha,valR[i-1], g, isclose(valR[i-1],g)])

['rosenbluth', None, 0.01, 0.009999999999999997, True]
['rosenbluth', None, 0.0149253731343284, 0.014925373134328354, True]
['rosenbluth', None, 0.0198990198990199, 0.019899019899019897, True]
['rosenbluth', None, 1.0, 0.999999999999988, True]
['rosenbluth', None, 0.0145995238085161, 0.014599523808516111, True]
['rosenbluth', None, 0.0154636114975739, 0.015463611497573776, True]


In [8]:
# polarization
for j in range(1,2):
    alpha = None
    if j==1:
        method = 'wolfson'
        valR = dfR.wolfson
            
    for i in range(1,nf+1):
        x = make_data(n,i) 
        df = pd.DataFrame({'x':x})+0.0  # si es entero tira error
        dr = ApodeData(df, varx="x")
        if alpha==None:
            g = dr.polarization(method)
        else:
            g = dr.polarization(method,alpha=alpha)
        print([method,alpha,valR[i-1], g, isclose(valR[i-1],g)])

['wolfson', None, 0.0, 4.39648317751562e-16, False]
['wolfson', None, 0.33009900990099, 0.3300990099009903, True]
['wolfson', None, 0.660133307194667, 0.6601333071946678, True]
['wolfson', None, inf, inf, True]
['wolfson', None, 0.29063747198598905, 0.2906374719859885, True]
['wolfson', None, 0.478014075557959, 0.4780140755579504, True]


D:/GitHub/apode\apode\polarization.py:70: RuntimeWarning: divide by zero encountered in double_scalars
  p_w = 4 * (0.5 - L - g/2) * (np.mean(ys) / np.median(ys))


### Test en Stata

In [ ]:
# Los metodos dan en general muy parecidos pero no igual
# Pueden ser variantes validas, o estar mal alguno
# pline=0  implica

In [4]:
n = 100
i = 1
df = pd.DataFrame({'y1':make_data(n,1),'y2':make_data(n,2),'y3':make_data(n,3),
                  'y4':make_data(n,4),'y5':make_data(n,5),'y6':make_data(n,6)})
df.to_csv("income.csv",index=False)

In [4]:
# leer datos de stata
# se puso -1 en el cvs cuando no había un número
fname = "poverty_stata.csv" 
path = 'D:\\GitHub\\apode\\Drafts\\test_R\\'
dfR = pd.read_csv(fname) 
nf = dfR.shape[0]
n = 100 # ver en R
dfR

,headcount,gap,watts,fgt2,fgt15,cuh01,cuh05,sen,thon,takayama
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,25,12.128713,22.717049,7.923243,9.604832,20.975924,15.916482,16.089109,22.154691,11.523944
2,35,23.308174,66.009064,18.545721,20.535610,56.229225,34.716400,28.907077,40.304134,22.382983
3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000
4,22,10.961053,21.793659,7.280053,8.746409,19.864923,14.608008,14.450521,20.220657,10.503494
5,34,14.213349,20.935888,7.512673,10.156595,20.054390,17.052786,18.300571,24.917643,12.602591


In [5]:
# poverty
# Stata no normaliza por poblacion

for j in range(1,11):
    alpha = None
    if j==1:
        method = 'headcount'
        valR = dfR.headcount
    elif j==2:
        method = 'gap'
        valR = dfR.gap
    elif j==3:
        method = 'watts'
        valR = dfR.watts
    elif j==4:
        method = 'severity'
        valR = dfR.fgt2
    elif j==5:
        method = 'fgt'
        valR = dfR.fgt15
        alpha = 1.5    
    elif j==6:
        method = 'cuh'
        valR = dfR.cuh01
        alpha = 0.1    
    elif j==7:
        method = 'cuh'
        valR = dfR.cuh05
        alpha = 0.5    
    elif j==8:
        method = 'sen'
        valR = dfR.sen
    elif j==9:
        method = 'thon'
        valR = dfR.thon      
    elif j==10:
        method = 'takayama'
        valR = dfR.takayama

    valR[valR==-1] = np.NaN    
    valR = valR/n # Stata no normaliza por poblacion    
    for i in range(1,nf+1):
        x = make_data(n,i) 
        df = pd.DataFrame({'x':x}) #+0.0  # si es entero tira error
        pline = np.median(x)/2
        dr = ApodeData(df, varx="x")
        if alpha==None:
            g = dr.poverty(method,pline=pline)
        else:
            g = dr.poverty(method,alpha=alpha,pline=pline)
        print([method,alpha,valR[i-1], g, isclose(valR[i-1],g)])

['headcount', None, 0.0, 0.0, True]
['headcount', None, 0.25, 0.25, True]
['headcount', None, 0.35, 0.35, True]
['headcount', None, 0.0, 0.0, True]
['headcount', None, 0.22, 0.22, True]
['headcount', None, 0.34, 0.34, True]
['gap', None, 0.0, 0.0, True]
['gap', None, 0.1212871265411377, 0.12128712871287128, False]
['gap', None, 0.23308174133300782, 0.23308174867673004, False]
['gap', None, 0.0, 0.0, True]
['gap', None, 0.10961052894592291, 0.10961053726796396, False]
['gap', None, 0.1421334934234619, 0.14213348850158591, False]
['watts', None, 0.0, 0.0, True]
['watts', None, 0.2271704864501953, 0.227170486700537, False]
['watts', None, 0.6600906372070314, 0.6600906188073208, False]
['watts', None, 0.0, 0.0, True]
['watts', None, 0.21793659210205077, 0.21793660145702864, False]
['watts', None, 0.2093588829040527, 0.20935889188201626, False]
['severity', None, 0.0, 0.0, True]
['severity', None, 0.07923243045806888, 0.07923242819331439, False]
['severity', None, 0.1854572105407715, 0.1854

C:\Programas\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
